In [1]:
from sqlalchemy import create_engine, Column, Integer, String, Boolean
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker


engine = create_engine('sqlite:///D:/python_scripts/Tasks/TODO_API/DB/TODO.db', echo=True)  # Change echo=True for logging SQL queries
Base = declarative_base()
class Todo(Base):
    __tablename__ = 'Tasks'
    id = Column(Integer, primary_key=True)
    title = Column(String(100))
    description = Column(String)
    completed = Column(Boolean, default=False)


C:\Users\nishantm\AppData\Local\Temp\ipykernel_35408\350767603.py:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [2]:
Base.metadata.create_all(engine)


2024-01-09 11:09:29,600 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-09 11:09:29,602 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Tasks")
2024-01-09 11:09:29,604 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-09 11:09:29,606 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Tasks")
2024-01-09 11:09:29,608 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-09 11:09:29,611 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Tasks" (
	id INTEGER NOT NULL, 
	title VARCHAR(100), 
	description VARCHAR, 
	completed BOOLEAN, 
	PRIMARY KEY (id)
)


2024-01-09 11:09:29,615 INFO sqlalchemy.engine.Engine [no key 0.00349s] ()
2024-01-09 11:09:29,620 INFO sqlalchemy.engine.Engine COMMIT


In [3]:
Session_OBJ = sessionmaker(bind=engine)
session = Session_OBJ()

In [4]:
new_todo = Todo(title="Task2", description="Task2")
session.add(new_todo)

In [5]:
from sqlalchemy import MetaData

metadata = MetaData()

# Reflect the existing database into MetaData
metadata.reflect(bind=engine)

# Drop all tables defined in the metadata
metadata.drop_all(bind=engine, checkfirst=True)

2024-01-09 11:09:36,216 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-09 11:09:36,218 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-01-09 11:09:36,219 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-09 11:09:36,221 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_temp_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-01-09 11:09:36,223 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-09 11:09:36,225 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("Tasks")
2024-01-09 11:09:36,226 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-09 11:09:36,228 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 'view')
2024-01-09 11:09:36,230 INFO sqlalchemy.engine.Engine [raw sql] ('Tasks',)
2024-01-09 11:09:36,232 INFO sqlalchemy.engine.Engin

In [1]:
import redis
redis_client = redis.StrictRedis(host='redis-12453.c325.us-east-1-4.ec2.cloud.redislabs.com', port=12453, decode_responses=True, password="B4AL1yNQ2GlEZcscZSQ3Xnsb3rudV3y7")
  

In [2]:
redis_client.

TypeError: ManagementCommands.echo() missing 1 required positional argument: 'value'